In [2]:
import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# ─────────────── USER PARAMETERS ───────────────
input_filepath  = r"Z:\UriMons\Fiber Photometry\adBNST_eEPA_GCaMP8s\Behavior\Grooming_Bins_Summary_10sec_All_Mice.xlsx"    # ← update this
rows_per_bin    = 300                          # rows per bin (e.g. 900)
bin_label       = "10 sec bin"                 # label for new column
# ──────────────────────────────────────────────────

# derive output path
base, ext = os.path.splitext(input_filepath)
output_filepath = f"{base}_highlighted{ext}"

# Step 1: read all sheets & compute metrics
# ------------------------------------------
all_sheets = pd.read_excel(input_filepath, sheet_name=None, engine="openpyxl")
processed = {}
for name, df in all_sheets.items():
    # find all "M..." columns
    mice_cols = [c for c in df.columns if str(c).startswith("M")]
    # compute row‐wise stats
    df["Average"]     = df[mice_cols].mean(axis=1)
    df["SD"]          = df[mice_cols].std(axis=1)
    df["avg per sec"] = df["Average"] / rows_per_bin
    df["SD per sec"]  = df["SD"] / rows_per_bin
    df["+2SD"]        = df["Average"] + 2 * df["SD"]
    df["-2SD"]        = df["Average"] - 2 * df["SD"]
    processed[name] = df

# write out all sheets with new columns
with pd.ExcelWriter(output_filepath, engine="openpyxl") as writer:
    for name, df in processed.items():
        df.to_excel(writer, sheet_name=name, index=False)

# Step 2: highlight outliers in lighter red across all sheets
# -----------------------------------------------------------
wb       = load_workbook(output_filepath)
light_red_fill = PatternFill(start_color="FFCCCCCC",
                             end_color="FFCCCCCC",
                             fill_type="solid")

for ws in wb.worksheets:
    # read headers from row 1
    headers     = [cell.value for cell in ws[1]]
    # find indices (1‑based) of mouse columns and ±2SD
    mice_idx    = [i+1 for i,h in enumerate(headers) if isinstance(h,str) and h.startswith("M")]
    plus2sd_idx = headers.index("+2SD") + 1
    minus2sd_idx= headers.index("-2SD") + 1

    # iterate data rows
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        plus_val  = row[plus2sd_idx-1].value
        minus_val = row[minus2sd_idx-1].value
        for col in mice_idx:
            cell = row[col-1]
            # check out‑of‑bounds
            if cell.value is not None and (cell.value > plus_val or cell.value < minus_val):
                cell.fill = light_red_fill

wb.save(output_filepath)
print(f"Done—highlighted workbook saved to:\n  {output_filepath}")


Done—highlighted workbook saved to:
  Z:\UriMons\Fiber Photometry\adBNST_eEPA_GCaMP8s\Behavior\Grooming_Bins_Summary_10sec_All_Mice_highlighted.xlsx
